In [74]:
# === Step 1: read data set ====================================================
import pandas as pd
df = pd.read_csv("intelligent-lighting.csv")
df.head()

,Light Sensor,Time,Light State
0,0.0,00:00,Off
1,0.0,01:00,Off
2,0.0,02:00,Off
3,0.0,03:00,Off
4,0.0,04:00,Off


In [75]:
# === Step 2: data preprocessing ===============================================
df_tmp = df.copy()
# group light sensor value into 4 levels
for i in range(len(df_tmp)):
    if (df_tmp.loc[i,'Light Sensor'] >= 0.0 and df_tmp.loc[i,'Light Sensor'] <= 0.25):
        df_tmp.loc[i,'Light Sensor'] = 0
    elif (df_tmp.loc[i,'Light Sensor'] >= 0.26 and df_tmp.loc[i,'Light Sensor'] <= 0.50):
        df_tmp.loc[i,'Light Sensor'] = 1
    elif (df_tmp.loc[i,'Light Sensor'] >= 0.51 and df_tmp.loc[i,'Light Sensor'] <= 0.75):
        df_tmp.loc[i,'Light Sensor'] = 2
    elif (df_tmp.loc[i,'Light Sensor'] >= 0.76 and df_tmp.loc[i,'Light Sensor'] <= 1.0):
        df_tmp.loc[i,'Light Sensor'] = 3
df_tmp['Light Sensor'] = df_tmp['Light Sensor'].astype('int64')    # convert float64 to int64
# get just the hour of the time
df_tmp['Time'] = pd.to_datetime(df_tmp['Time'], format='%H:%M').dt.hour
# encode 'Off' to 0 and 'On' to 1
df_tmp.replace({'Light State': {'Off': 0, 'On': 1}}, inplace=True)
df_tmp.head()

,Light Sensor,Time,Light State
0,0,0,0
1,0,1,0
2,0,2,0
3,0,3,0
4,0,4,0


In [76]:
# === Step 3: convert pandas data frame to numpy array =========================
df_np = df_tmp.to_numpy()
# print(df_np)
# split the data into features and label
X = df_np[:, 0:2]    # features
y = df_np[:, 2]    # label

In [77]:
# === Step 4: create frequency table for parameter light sensor ================
import numpy as np
freq_table_lsensor = np.zeros((4, 2))
for i in range(len(y)):
    freq_table_lsensor[X[i, 0], y[i]] += 1
print(freq_table_lsensor)

[[ 6. 12.]
 [ 4.  5.]
 [ 5.  0.]
 [ 5.  0.]]


In [78]:
# === Step 5: create frequency table for parameter time ========================
freq_table_time = np.zeros((24, 2))
for i in range(len(y)):
    freq_table_time[X[i, 1], y[i]] += 1
print(freq_table_time)

[[1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [2. 0.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [1. 1.]
 [1. 1.]
 [1. 0.]
 [1. 0.]
 [2. 1.]
 [2. 0.]
 [1. 1.]
 [1. 1.]
 [1. 0.]
 [1. 1.]
 [1. 1.]
 [1. 1.]
 [0. 2.]
 [0. 2.]
 [0. 1.]
 [0. 1.]]


In [79]:
# === Step 6: create function for calculating label prior probability ==========
def label_prior_prob(light_state):
    count = 0
    for i in range(len(y)):
        if (y[i] == light_state):
            count += 1
    prob = count/len(y)
    prob_lap = (count+1)/(len(y)+2*1)
    return prob_lap

# test the function
label_prior_prob(0)    # probability of 'Off'

0.5384615384615384

In [80]:
# === Step 7: create function for calculating likelihood =======================
def likelihood(light_sensor, time, light_state):
    # calculate probability of light sensor=light_sensor given light state=light_state
    prob_lsensor_lstate = freq_table_lsensor[light_sensor, light_state] / np.sum(freq_table_lsensor[:, light_state])
    prob_lsensor_lstate_lap = (freq_table_lsensor[light_sensor, light_state]+1) / (np.sum(freq_table_lsensor[:, light_state])+4*1)
    # calculate probability of time=time given light state=light_state
    prob_time_lstate = freq_table_time[time, light_state] / np.sum(freq_table_time[:, light_state])
    prob_time_lstate_lap = (freq_table_time[time, light_state]+1) / (np.sum(freq_table_time[:, light_state])+24*1)
    # calculate probability of light sensor=light_sensor and time=time given light state=light_state
    prob_lsensor_time_lstate = prob_lsensor_lstate * prob_time_lstate
    prob_lsensor_time_lstate_lap = prob_lsensor_lstate_lap * prob_time_lstate_lap
    return prob_lsensor_time_lstate_lap

# test the function
likelihood(2, 12, 0)    # likelihood of light sensor level 2 and time = 12:00 given light state 'Off'

0.017045454545454544

In [81]:
# === Step 8: create function for calculating posterior probability ============
def posterior_prob(light_sensor, time, light_state):
    prob = (likelihood(light_sensor, time, light_state) * label_prior_prob(light_state))
    return prob

In [82]:
# === Step 9: make prediction =================================================
# probability of light state 'Off' given light sensor level 2 and time = 12:00 
print(posterior_prob(2, 12, 0))
# probability of light state 'On' given light sensor level 2 and time = 12:00 
print(posterior_prob(2, 12, 1))

0.009178321678321678
0.0010720986330742428


In [83]:
# === Step 11: print features as C array =======================================
print("uint8_t X[%d][%d] = {" %(np.shape(X)[0], np.shape(X)[1]), end='')
for i in range(np.shape(X)[0]):    # rows
    print("{", end='')
    for j in range(np.shape(X)[1]):    # columns
        if (j == (np.shape(X)[1]-1)):
            print("%d" %(X[i,j]), end='')
        else:
            print("%d," %(X[i,j]), end='')
    if (i == (np.shape(X)[0]-1)):
        print("}", end='')
    else:
        print("},", end='')
print("};")

uint8_t X[37][2] = {{0,0},{0,1},{0,2},{0,3},{0,4},{0,4},{0,5},{0,6},{1,7},{1,8},{0,8},{1,9},{1,9},{2,10},{2,11},{3,12},{3,12},{1,12},{3,13},{3,13},{3,14},{1,14},{2,15},{0,15},{2,16},{2,17},{0,17},{1,18},{1,18},{1,19},{0,19},{0,20},{0,20},{0,21},{0,21},{0,22},{0,23}};


In [84]:
# === Step 12: print label as C array ==========================================
print("uint8_t y[%d] = {" %(np.shape(y)[0]), end='')
for i in range(np.shape(y)[0]):
    if (i == (np.shape(y)[0]-1)):
        print("%d" %(y[i]), end='')
    else:
        print("%d," %(y[i]), end='')
print("};")

uint8_t y[37] = {0,0,0,0,0,0,1,1,1,0,1,0,1,0,0,0,0,1,0,0,0,1,0,1,0,0,1,0,1,0,1,1,1,1,1,1,1};
